In [1]:
# importing modules
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
from datetime import date, timedelta, datetime
import time
import re

In [2]:
# Set up the web driver (provide the path to your Firefox web driver)
chrome_options = webdriver.FirefoxOptions()

chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--verbose")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

driver = webdriver.Firefox(options=chrome_options)

driver.get("https://www.arise.tv")

In [3]:
# Clicking the search icon
search_icon = driver.find_element(By.XPATH, "//li[@class='pos-relative search-toggle']")
search_icon.click()

In [4]:
# Get the input elements
search_input = driver.find_element(By.XPATH, "//input[@class='search-field']")
search_button = driver.find_element(By.XPATH, "//input[@class='search-submit']")

In [5]:
# Send the input to the webpage
search_input.send_keys('nigeria')
time.sleep(1)
search_button.click()

In [6]:
# Define today's date and yesterday's date

today = date.today()
yesterday = today - timedelta(days=1)

def post_date_check(post_date):
    date_match = re.search(r'(\d{1,2})(?:st|nd|rd|th)?\s(\w+),\s(\d{4})', post_date)
    
    if date_match:
        day, month, year = date_match.groups()
        month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                     'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
        month_number = month_map.get(month)
        article_date = datetime(int(year), month_number, int(day)).date()

        return article_date == today or article_date == yesterday

    return False


In [7]:
post_date_check('24th Oct, 2023')

True

In [11]:
todays_and_yesturdays_article_urls = []

# collecting article's url from Exclusives & Trending section
article_list_2 = driver.find_elements(By.XPATH, "//div[@class='panel']//article")
for article in article_list_2:
    date_text = article.find_element(By.CLASS_NAME, "date").text
    date_match = post_date_check(date_text)
 
    if date_match:
        a_link = article.find_element(By.XPATH, "div[@class='article-content']//h3//a").get_attribute('href')
        # print(a_link)
        if a_link not in todays_and_yesturdays_article_urls:
            todays_and_yesturdays_article_urls.append(a_link)
            

In [48]:
# Scrapping articles with pagination
new_articles = True

# Loop through the pages
while new_articles:
    
    # Scrolling down to load more search results
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
    
    # collecting all articles from the current page
    article_list = driver.find_elements(By.XPATH, "//div[@id='arise_loop_content']//article")
    
    # Extract the URLs of today's and yesterday's articles 
    for article in article_list:
        # Extract the date
        date_text = article.find_element(By.CLASS_NAME, "date").text
        date_match = post_date_check(date_text)

        if date_match:
            a_link = article.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a').get_attribute('href')

            if a_link not in todays_and_yesturdays_article_urls:
                todays_and_yesturdays_article_urls.append(a_link)
        else:
            new_articles = False
            # break
        
    # Find and click the pagination link for the next page
    try:
        next_page_link = driver.find_element(By.XPATH, "//div[@class='nav-previous']")
        next_page_link.click()
    except NoSuchElementException:
        print("No 'Next Page' link found. Exiting the loop.")
        break  # Exit the loop if there is no "Next Page" link

In [49]:
len(todays_and_yesturdays_article_urls)

11

In [61]:
# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["Headline", "Author", "Post_date", "Content"])

for article_url in todays_and_yesturdays_article_urls:
    driver.get(article_url)
    post_date = driver.find_element(By.CLASS_NAME, "date").text
    
    # collecting the data
    headline = driver.find_element(By.XPATH, "//h1[@class='article-title mb-2']").text
    author = driver.find_element(By.CLASS_NAME, "author").text
    p_content = driver.find_elements(By.XPATH, "//div[@class='article-content-content']//p")
    content = ''
    for i in p_content:
        content += i.text
        
    content = content.replace("    ", " ")
    
    # Storing the date in to dataframe
    df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_5608\2835225190.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5608\2835225190.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5608\2835225190.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Headline": headline, "Author": author, "Post_date": post_date, "Content": content}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\

In [62]:
len(df)

11

In [65]:
# df.sample(2)

In [66]:
df['Headline']

0                                           Bishop Kukah On Israel-Hamas War: It’s Dangerous To Use Identity As A Weapon
1                                          INEC Appointments Should Be Independent of Presidency, Says Former Chair Jega
2                                          Aondoakaa: Nigeria Should Take Action Against Corrupt Schemes Similar To P&ID
3                                               Tinubu-Led FEC Approves $5BN Poverty Trust Fund – Trending With Ojy Okpe
4                Nigeria’s Senate Constitutes Panel To Probe NNPC Over N11.35tn Spent On Moribund Refineries In 13 Years
5     Former Nigerian President Jonathan Insists the Problem of Africa is the Making of Leaders Who Fail to Respect Laws
6                                               Tax Reforms: Nigeria To Streamline Over 200 Taxes and Levies To Under 10
7                                           First Lady Remi Tinubu Bemoans High Prevalence Of Cervical Cancer In Nigeria
8                               

In [ ]:
# storing data in to excel file
df.to_excel('arise_tv_article.xlsx', index=False)

In [57]:
driver.back()

In [67]:
driver.quit()